# Data processing of OpenSky COVID-19 Flight Dataset

Import libraries

In [1]:
from pathlib import Path
import pandas as pd

Read all OpenSky CSV files (takes some time)

In [ ]:
flightlist = pd.concat(
    pd.read_csv(file, parse_dates=["firstseen", "lastseen", "day"])
    for file in Path("opensky_data").glob("flightlist_*.csv.gz")
)
flightlist = flightlist.drop(columns=["Unnamed: 0"])

Basic exploratory data analysis

In [ ]:
flightlist.head()

In [ ]:
flightlist.describe()

In [ ]:
print(len(flightlist))
print(flightlist.origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.destination.isnull().sum(axis = 0) / len(flightlist))

Delete all records that have no origin or destination data (this action is subject to discussion)

In [ ]:
flightlist = flightlist[flightlist['origin'].notna() & flightlist['destination'].notna()]

### Make matching of countries by ICAO codes table

Read ICAO data

In [ ]:
icao_codes = pd.read_excel(Path("auxiliary_data/icao_codes.xlsx"), usecols=["ICAO", "Country"], sheet_name = "data")

In [ ]:
icao_codes.head()

Join by origin

In [ ]:
flightlist = pd.merge(flightlist, icao_codes, how="left", left_on = ["origin"], right_on=["ICAO"])
flightlist = flightlist.rename(columns={"Country": "country_origin"}).drop(columns=["ICAO"])

Join by destination

In [ ]:
flightlist = pd.merge(flightlist, icao_codes, how="left", left_on = ["destination"], right_on=["ICAO"])
flightlist = flightlist.rename(columns={"Country": "country_destination"}).drop(columns=["ICAO"])

Explore results of join

In [ ]:
flightlist.head()

In [ ]:
print(len(flightlist))
print(flightlist.country_origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.country_destination.isnull().sum(axis = 0) / len(flightlist))

### Make matching of countries by ICAO nationality prefixes

Create prefixes for origin and destination

In [ ]:
flightlist['origin_pref'] = flightlist['origin'].astype(str).str[:2]
flightlist['destination_pref'] = flightlist['destination'].astype(str).str[:2]
flightlist.head()

Read ICAO nationality prefixes data

In [ ]:
country_prefix = pd.read_excel(Path("auxiliary_data/icao_nationality_prefixes.xlsx"), usecols=["prefix", "Country"], sheet_name = "data")

In [ ]:
country_prefix.head()

Join by origin and destination (1st and 2nd cells respectively)

In [ ]:
flightlist = pd.merge(flightlist, country_prefix, how="left", left_on = ["origin_pref"], right_on=["prefix"])
flightlist = flightlist.rename(columns={"Country": "country_origin_2"}).drop(columns=["prefix"])

In [ ]:
flightlist = pd.merge(flightlist, country_prefix, how="left", left_on = ["destination_pref"], right_on=["prefix"])
flightlist = flightlist.rename(columns={"Country": "country_destination_2"}).drop(columns=["prefix"])

In [ ]:
flightlist.head()

Fill NAs from the 1st join with values from the 2nd join

In [ ]:
flightlist['country_origin'] = flightlist['country_origin'].fillna(flightlist['country_origin_2'])
flightlist['country_destination'] = flightlist['country_destination'].fillna(flightlist['country_destination_2'])

In [ ]:
print(len(flightlist))
print(flightlist.country_origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.country_destination.isnull().sum(axis = 0) / len(flightlist))

Remove records which didn't match with any country

In [ ]:
flightlist = flightlist[flightlist['country_origin'].notna() & flightlist['country_destination'].notna()]

Delete unnecessary columns

In [ ]:
flightlist = flightlist.drop(columns=["country_origin_2", "country_destination_2", "origin_pref", "destination_pref"])

Add a column that indicates whether the flight is international or not.

In [ ]:
flightlist['international'] = (flightlist["country_origin"] != flightlist["country_destination"])

Remove timezone information from "day" column

In [ ]:
flightlist['day'] = flightlist['day'].dt.date

Calculate the number of domestic flights for every country by dates

In [ ]:
domestic_flights=flightlist[~flightlist.international].groupby(['country_origin', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'domestic_flights', 'country_origin':'country'})

Calculate the number of international departures and arrivals for every country by dates

In [ ]:
international_departures=flightlist[flightlist.international].groupby(['country_origin', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'international_departures', 'country_origin':'country'})

In [ ]:
international_arrivals=flightlist[flightlist.international].groupby(['country_destination', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'international_arrivals', 'country_destination':'country'})

Join all these tables

In [ ]:
flights = pd.merge(domestic_flights, international_departures, how="outer", on=["country", "day"])

In [ ]:
flights = pd.merge(flights, international_arrivals, how="outer", on=["country", "day"])

In [ ]:
flights['total_flights'] = flights['domestic_flights'] + flights['international_departures'] + flights['international_arrivals']

In [ ]:
flights.head()

Write results to CSV and Excel

In [ ]:
flights.to_csv(Path("flight_data/flights.csv"), index=False)

In [ ]:
flights.to_excel(Path("flight_data/flights.xlsx"), index=False)